In [21]:
import numpy as np 
import random 
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
data = pd.read_csv("/content/Restart Quit form (Responses) - Form Responses 1.csv")
levels = {
    "level_one":1,
    "level_two":2,
    "level_three":3,
    "level_four":4,
    "level_five":5,
    "level_six":6,
    "level_seven":7
}
data = data.iloc[344:]
data['Level'] = data['Level'].apply(lambda x: levels[x])
data.drop('Timestamp',axis=1,inplace=True)
data = data.groupby(['Level','Restart_Quit']).size().reset_index()
data.columns = ['Level','Status','Count']
data

In [37]:
fig = px.bar(data, x="Level", y="Count",
             color='Status', barmode='group',text="Count",
             height=400)
fig.show()

In [69]:
analytics_data = pd.read_csv("/content/Analytics Form (Responses) - Form Responses 1 (1).csv")
analytics_data = analytics_data.iloc[1670:]

In [70]:
analytics_data.columns

Index(['Timestamp', 'Distance', 'Hits', 'Misses', 'Level', 'Score',
       'ScoreLossPowerups', 'ScoreLoss', 'ScoreGain', 'LaneSwitches',
       'ReachedEnd', 'powerups_pickedup', 'powerups_used', 'Score.1',
       'Score.2'],
      dtype='object')

In [79]:
# Metrics on Lane Switches
lane_switch_data = analytics_data[['LaneSwitches','Level']]
lane_switch_data = lane_switch_data[lane_switch_data.Level.isin(levels.keys())]
lane_switch_data = lane_switch_data.groupby(['Level']).agg({'LaneSwitches':['max','mean']}).reset_index()
lane_switch_data.columns = ['Level','Max_Laneswitches','Mean_Laneswitches']
lane_switch_data['Level'] = lane_switch_data['Level'].apply(lambda x:levels[x])
rows = []
for index,row in lane_switch_data.iterrows():
  level = row['Level'] 
  max_switches = round(row['Max_Laneswitches'],2)
  mean_switches = round(row['Mean_Laneswitches'],2)
  row1 = [level,"MAX_SWITCHES",max_switches]
  row2 = [level,"MEAN_SWITCHES",mean_switches]
  rows.append(row1)
  rows.append(row2)
new_dataframe = pd.DataFrame(rows,columns = ['Level','Status','Count'])
fig = px.bar(new_dataframe, x="Level", y="Count",
             color='Status', barmode='group',text="Count",
             height=400)
fig.show()

In [85]:
# Reached end or not
analytics_data = pd.read_csv("/content/Analytics Form (Responses) - Form Responses 1 (1).csv")
analytics_data = analytics_data.iloc[1670:]
reached_end_data = analytics_data[['ReachedEnd','Level']]
reached_end_data = reached_end_data[reached_end_data.ReachedEnd.isin(['Yes','No'])]
reached_end_data = reached_end_data.groupby(['Level','ReachedEnd']).agg({'ReachedEnd':['count']}).reset_index()
reached_end_data.columns = ['Level','Status','Count']
reached_end_data['Level'] = reached_end_data['Level'].apply(lambda x:levels[x])
fig = px.bar(reached_end_data, x="Level", y="Count",
             color='Status', barmode='group',text="Count",
             height=400)
fig.show()

In [93]:
# Score gained, score lost per level 
# 'Score', 'ScoreLossPowerups', 'ScoreLoss', 'ScoreGain'
analytics_data = pd.read_csv("/content/Analytics Form (Responses) - Form Responses 1 (1).csv")
analytics_data = analytics_data.iloc[1670:]
score_data = analytics_data[['Level','Score','ScoreLoss','ScoreGain']]
score_data.dropna(inplace=True)
score_data = score_data.groupby(['Level']).agg({'ScoreLoss':'mean','ScoreGain':'mean','Score':['max','mean']}).reset_index()
score_data.columns = ['Level','MeanScoreLoss','MeanScoreGain','MaxScore','MeanScore']
score_data['Level'] = score_data['Level'].apply(lambda x:levels[x])
rows = []
for index,row in score_data.iterrows():
  row1 = [row['Level'],'Mean Score Loss',round(row['MeanScoreLoss'],1)]
  row2 = [row['Level'],'Mean Score Gain',round(row['MeanScoreGain'],1)]
  row3 = [row['Level'],'Max Score',round(row['MaxScore'],1)]
  row4 = [row['Level'],'Mean Score',round(row['MeanScore'],1)]
  rows.append(row1)
  rows.append(row2)
  rows.append(row3)
  rows.append(row4)

new_dataframe = pd.DataFrame(rows,columns = ['Level','Status','Score'])
fig = px.bar(new_dataframe, x="Level", y="Score",
             color='Status', barmode='group',text="Score",
             height=400)
fig.show()

In [112]:
# Hits and Misses per level 
analytics_data = pd.read_csv("/content/Analytics Form (Responses) - Form Responses 1 (1).csv")
analytics_data = analytics_data.iloc[1670:]
hits_and_misses_data = analytics_data[['Level','Hits','Misses']]
hits_and_misses_data = hits_and_misses_data[hits_and_misses_data.Level.isin(levels.keys())]
hits_and_misses_data['Level'] = hits_and_misses_data['Level'].apply(lambda x:levels[x])

hits_and_misses_data['hits_first'] = hits_and_misses_data['Hits'].apply(lambda x:int(x.split(" ")[0]))
hits_and_misses_data['hits_middle'] = hits_and_misses_data['Hits'].apply(lambda x:int(x.split(" ")[1]))
hits_and_misses_data['hits_last'] = hits_and_misses_data['Hits'].apply(lambda x:int(x.split(" ")[2]))

hits_and_misses_data['miss_first'] = hits_and_misses_data['Misses'].apply(lambda x:int(x.split(" ")[0]))
hits_and_misses_data['miss_middle'] = hits_and_misses_data['Misses'].apply(lambda x:int(x.split(" ")[1]))
hits_and_misses_data['miss_last'] = hits_and_misses_data['Misses'].apply(lambda x:int(x.split(" ")[2]))

hits_and_misses_data.drop(['Hits','Misses'],axis=1,inplace=True)

hits_and_misses_data = hits_and_misses_data.groupby(['Level']).agg({'hits_first':'mean','hits_middle':'mean','hits_last':'mean',
                                                                    'miss_first':'mean','miss_middle':'mean','miss_last':'mean'}).reset_index()

hits_and_misses_data['hits_first'] = hits_and_misses_data['hits_first'].apply(lambda x:round(x,1))
hits_and_misses_data['hits_middle'] = hits_and_misses_data['hits_middle'].apply(lambda x:round(x,1))
hits_and_misses_data['hits_last'] = hits_and_misses_data['hits_last'].apply(lambda x:round(x,1))
hits_and_misses_data['miss_first'] = hits_and_misses_data['miss_first'].apply(lambda x:round(x,1))
hits_and_misses_data['miss_middle'] = hits_and_misses_data['miss_middle'].apply(lambda x:round(x,1))
hits_and_misses_data['miss_last'] = hits_and_misses_data['miss_last'].apply(lambda x:round(x,1))

hits_and_misses_data
rows = []
for index,row in hits_and_misses_data.iterrows():
  level = row['Level']

  first_hit = row['hits_first']
  middle_hit = row['hits_middle']
  last_hit = row['hits_last']

  first_miss = row['miss_first']
  middle_miss = row['miss_middle']
  last_miss = row['miss_last']

  row1 = [level,"First",first_hit,"Hits"]
  row2 = [level,"Middle",first_hit,"Hits"]
  row3 = [level,"Last",first_hit,"Hits"]

  row4 = [level,"First",first_miss,"Miss"]
  row5 = [level,"Middle",middle_miss,"Miss"]
  row6 = [level,"Last",last_miss,"Miss"]

  rows.append(row1)
  rows.append(row2)
  rows.append(row3)
  rows.append(row4)
  rows.append(row5)
  rows.append(row6)

df = pd.DataFrame(rows,columns = ['c1','c2','values','DF'])
alt.Chart(df).mark_bar().encode(

    # tell Altair which field to group columns on
    x=alt.X('c2:N', title=None),

    # tell Altair which field to use as Y values and how to calculate
    y=alt.Y('sum(values):Q',
        axis=alt.Axis(
            grid=False,
            title=None)),

    # tell Altair which field to use to use as the set of columns to be  represented in each group
    column=alt.Column('c1:N', title=None),

    # tell Altair which field to use for color segmentation 
    color=alt.Color('DF:N',
            scale=alt.Scale(
                # make it look pretty with an enjoyable color pallet
                range=['#96ceb4', '#ffcc5c','#ff6f69'],
            ),
        ))\
    .configure_view(
        # remove grid lines around column clusters
        strokeOpacity=0    
    )#c1,c2,values,DF

alt.Chart(...)

In [114]:
def add_powerups(powerup):
  powerup = powerup.split(",")
  powerup = [int(x) for x in powerup]
  return sum(powerup)

In [118]:
# Total power ups picked vs total power ips used per level 
analytics_data = pd.read_csv("/content/Analytics Form (Responses) - Form Responses 1 (1).csv")
analytics_data = analytics_data.iloc[1670:]
powerups_data = analytics_data[['Level','powerups_pickedup','powerups_used']]
powerups_data = powerups_data[powerups_data.Level.isin(levels.keys())]
powerups_data['Level'] = powerups_data['Level'].apply(lambda x:levels[x])
powerups_data['powerups_pickedup'] = powerups_data['powerups_pickedup'].apply(lambda x:add_powerups(x))
powerups_data['powerups_used'] = powerups_data['powerups_used'].apply(lambda x:add_powerups(x))

powerups_data = powerups_data.groupby(['Level']).agg({'powerups_pickedup':'mean','powerups_used':'mean'}).reset_index()
rows = []

for index,row in powerups_data.iterrows():
  level = row['Level']
  picked_up = round(row['powerups_pickedup'],1)
  used = round(row['powerups_used'],1)
  row1 = [level,'PICKED',picked_up]
  row2 = [level,'USED',used]
  rows.append(row1)
  rows.append(row2)

new_dataframe = pd.DataFrame(rows,columns = ['Level','Status','Mean'])

fig = px.bar(new_dataframe, x="Level", y="Mean",
             color='Status', barmode='group',text="Mean",
             height=400)
fig.show()
